In [57]:
import pandas as pd
import ast  # For safely evaluating string-formatted lists
import plotly.express as px

arenas_dict = {  
       54000002:      "Arena 1",                      
       54000003:      "Arena 2",                  
       54000004:      "Arena 3",                  
       54000005:      "Arena 4",                  
       54000006:      "Arena 5",                  
       54000008:      "Arena 6",                  
       54000009:      "Arena 7",                  
       54000010:      "Arena 8",                  
       54000007:      "Arena 9",                  
       54000024:      "Arena 10",            
       54000011:      "Arena 11",                  
       54000055:      "Arena 12",                  
       54000056:      "Arena 13",                  
       54000012:      "Arena 14",                  
       54000013:      "Arena 15",                  
       54000014:      "Arena 16",                  
       54000015:      "Arena 17",                  
       54000016:      "Arena 18",                  
       54000017:      "Arena 19",                  
       54000018:      "Arena 20",                  
       54000019:      "Arena 21",                  
       54000020:      "Arena 22",                  
       54000031:      "Arena 23",                
       54000117:      "Arena 24",     
}

# --- Helper Function ---
def check_for_card(card_list_str, card_name):
    """
    Safely parses a string representation of a card list
    and checks if a specific card is present.
    """
    try:
        # ast.literal_eval is safer than eval()
        card_list = ast.literal_eval(card_list_str)
        if not isinstance(card_list, list):
            return False
        
        # The list contains tuples like ('Card Name', level, 0)
        for card_tuple in card_list:
            if isinstance(card_tuple, tuple) and len(card_tuple) > 0 and card_tuple[0] == card_name:
                return True
    except (ValueError, SyntaxError, TypeError):
        # Handles cases where the string is not a valid list (e.g., NaN, malformed)
        return False
    return False

def plot_arena_card_win_loss(card_name):
    try:
        # Load the dataset
        file_name = '../#2 Data Storage/Processed Data/preprocessed_battle_log.csv'
        df = pd.read_csv(file_name)
        CARD_TO_FIND = card_name

        print(f"Data loaded. Processing for '{CARD_TO_FIND}' ...")

        # Initialize list to store results
        card_data = []

        # Iterate through all battles
        for row in df.itertuples():
            # Check player 0
            player_0_has_card = check_for_card(row.players_0_spells, CARD_TO_FIND)
            if player_0_has_card:
                card_data.append({
                    'arena': row.arena,
                    'outcome': 'Won' if row.players_0_winner == 1 else 'Lost'
                })
            
            # Check player 1
            player_1_has_card = check_for_card(row.players_1_spells, CARD_TO_FIND)
            if player_1_has_card:
                card_data.append({
                    'arena': row.arena,
                    'outcome': 'Won' if row.players_1_winner == 1 else 'Lost'
                })

        # Create a new DataFrame
        if not card_data:
            print(f"No usage data found for '{CARD_TO_FIND}'.")
        else:
            card_df = pd.DataFrame(card_data)
            
            # Group data by arena and outcome to get counts
            grouped_df = card_df.groupby(['arena', 'outcome']).size().reset_index(name='count')
            grouped_df['arena'] = grouped_df['arena'].map(arenas_dict)  # Map arena IDs to names
            grouped_df = grouped_df.dropna(subset=['arena'])

            arena_order = sorted(list(set(grouped_df['arena'])), key=lambda x: int(x.split(' ')[1]))
            grouped_df['arena'] = pd.Categorical(grouped_df['arena'], categories=arena_order, ordered=True)
            grouped_df = grouped_df.sort_values(by='arena')

            print(f"\nAggregated {CARD_TO_FIND} Usage Data:")
            print(grouped_df.to_string(index=False))

            color_map = {
            'Won': 'blue',
            'Lost': 'red'
            }
            
            # Create the visualization using Plotly Express
            fig = px.bar(
                grouped_df,
                x='arena',
                y='count',
                color='outcome',
                title=f'{CARD_TO_FIND} Usage Count per Arena by Win/Loss',
                labels={
                    'arena': 'Arena',
                    'count': f'{CARD_TO_FIND} Usage Count',
                    'outcome': f'Outcome for User'
                },
                barmode='overlay',
                color_discrete_map=color_map
            )
            
            fig.update_xaxes(categoryorder='array', categoryarray=arena_order)
            fig.show()
            
            print(f"{CARD_TO_FIND} Arena-wise Win Rate:")
            for arena in arena_order:
                arena_win_count = grouped_df[(grouped_df['outcome'] == 'Won') & (grouped_df['arena'] == arena)]['count'].sum()
                arena_lose_count = grouped_df[(grouped_df['outcome'] == 'Lost') & (grouped_df['arena'] == arena)]['count'].sum()

                arena_win_rate = arena_win_count/(arena_win_count + arena_lose_count)
                print(f"{arena}, Win Rate: {(arena_win_rate*100):.2f}")
            
            win_count = grouped_df[grouped_df['outcome'] == 'Won']['count'].sum()
            lost_count = grouped_df[grouped_df['outcome'] == 'Lost']['count'].sum()
            
            win_rate = win_count / (win_count + lost_count)
            print(f"\n{CARD_TO_FIND} global win rate: {(win_rate*100):.2f}%")
            
    except FileNotFoundError:
        print(f"Error: The file '{file_name}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [58]:
plot_arena_card_win_loss('Mega Knight')

Data loaded. Processing for 'Mega Knight' ...

Aggregated Mega Knight Usage Data:
   arena outcome  count
 Arena 3     Won      2
 Arena 4     Won      2
 Arena 5    Lost      1
 Arena 6     Won      7
 Arena 6    Lost      5
 Arena 7    Lost      1
 Arena 7     Won      3
 Arena 8    Lost      2
 Arena 8     Won      3
 Arena 9    Lost      1
 Arena 9     Won      2
Arena 10    Lost     93
Arena 10     Won    118
Arena 11    Lost    163
Arena 11     Won    236
Arena 12     Won    594
Arena 12    Lost    459
Arena 13    Lost    435
Arena 13     Won    574
Arena 14    Lost    369
Arena 14     Won    495
Arena 15    Lost    270
Arena 15     Won    304
Arena 16    Lost    350
Arena 16     Won    428
Arena 17    Lost    315
Arena 17     Won    368
Arena 18     Won    269
Arena 18    Lost    268
Arena 19     Won    122
Arena 19    Lost     88
Arena 20     Won    122
Arena 20    Lost    101
Arena 21    Lost    128
Arena 21     Won    164
Arena 22    Lost    167
Arena 22     Won    186
Arena 

Mega Knight Arena-wise Win Rate:
Arena 3, Win Rate: 100.00
Arena 4, Win Rate: 100.00
Arena 5, Win Rate: 0.00
Arena 6, Win Rate: 58.33
Arena 7, Win Rate: 75.00
Arena 8, Win Rate: 60.00
Arena 9, Win Rate: 66.67
Arena 10, Win Rate: 55.92
Arena 11, Win Rate: 59.15
Arena 12, Win Rate: 56.41
Arena 13, Win Rate: 56.89
Arena 14, Win Rate: 57.29
Arena 15, Win Rate: 52.96
Arena 16, Win Rate: 55.01
Arena 17, Win Rate: 53.88
Arena 18, Win Rate: 50.09
Arena 19, Win Rate: 58.10
Arena 20, Win Rate: 54.71
Arena 21, Win Rate: 56.16
Arena 22, Win Rate: 52.69
Arena 23, Win Rate: 52.36

Mega Knight global win rate: 55.27%
